# A simple sentiment prototype

## [github.com/williamsmj/strata-2015](http://github.com/williamsmj/strata-2015)

In [1]:
import os  # manipulate paths
import pandas as pd  # SQL-like operations and convenience functions
import joblib  # save and load models

Download the Sentiment140 data from [their website](http://help.sentiment140.com/for-students) and set `DATA_DIR` to the directory in which you have put the `CSV` files.

In [2]:
DATA_DIR = "trainingandtestdata"
training_csv_file = os.path.join(DATA_DIR, 'training.1600000.processed.noemoticon.csv')

## A peek at the data

In [3]:
names = ('polarity', 'id', 'date', 'query', 'author', 'text')
df = pd.read_csv(training_csv_file, encoding='latin1', names=names)

In [5]:
pd.options.display.max_colwidth = 140  # allow wide columns
df.head()  # show first 5 rows

,polarity,id,date,query,author,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


In [6]:
df.tail()

,polarity,id,date,query,author,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best feeling ever
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interviews! â« http://blip.fm/~8bmta
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me for details
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H


In [7]:
df['polarity'].replace({0: -1, 4: 1}, inplace=True)
text = df['text']
target = df['polarity'].values

In [8]:
print(len(target), len(text))

1600000 1600000


## Train the model

Set 20% of the data aside to test the trained model

In [9]:
from sklearn.cross_validation import train_test_split

text_train, text_validation, target_train, target_validation = (
    train_test_split(text, target, test_size=0.2, random_state=42)
)

Build a pipeline

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=100000)
feature_selector = SelectKBest(chi2, k=5000)
classifier = LogisticRegressionCV(n_jobs=4)

sentiment_pipeline = Pipeline((
    ('v', vectorizer),
    ('f', feature_selector),
    ('c', classifier)
))

This next cell took ~3 minutes to run on my machine

In [11]:
%time sentiment_pipeline.fit(text_train, target_train)
joblib.dump(sentiment_pipeline, 'model.pkl');

CPU times: user 1min 20s, sys: 5.88 s, total: 1min 26s
Wall time: 3min 26s


['model.pkl',
 'model.pkl_01.npy',
 'model.pkl_02.npy',
 'model.pkl_03.npy',
 'model.pkl_04.npy',
 'model.pkl_05.npy',
 'model.pkl_06.npy',
 'model.pkl_07.npy',
 'model.pkl_08.npy',
 'model.pkl_09.npy',
 'model.pkl_10.npy']

In [12]:
sentiment_pipeline = joblib.load('model.pkl')

## Test the model

In [13]:
for text, target in zip(text_validation[:10], target_validation[:10]):
    print(sentiment_pipeline.predict([text])[0], target, '\t', text)

1 -1 	 @chrishasboobs AHHH I HOPE YOUR OK!!! 
1 -1 	 @misstoriblack cool , i have no tweet apps  for my razr 2
1 -1 	 @TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
-1 -1 	 School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
1 -1 	 upper airways problem 
1 -1 	 Going to miss Pastor's sermon on Faith... 
1 1 	 on lunch....dj should come eat with me 
-1 -1 	 @piginthepoke oh why are you feeling like that? 
-1 -1 	 gahh noo!peyton needs to live!this is horrible 
1 1 	 @mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


In [14]:
sentiment_pipeline.score(text_validation, target_validation)

0.80733750000000004

## What did the model learn?

In [15]:
feature_names = sentiment_pipeline.steps[0][1].get_feature_names()
feature_names = [feature_names[i] for i in 
                 sentiment_pipeline.steps[1][1].get_support(indices=True)]

def show_most_informative_features(feature_names, clf, n=1000):
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [16]:
show_most_informative_features(feature_names, sentiment_pipeline.steps[2][1], n=500)

	-3.3584	not happy      		2.6298	no problem     
	-3.1676	clean me       		2.5395	no worries     
	-2.9222	not looking    		2.3611	cannot wait    
	-2.8815	sad            		2.2642	cant wait      
	-2.7330	inaperfectworld		2.2025	never too      
	-2.6631	passed away    		2.1215	not bad        
	-2.6604	sadly          		2.0880	bad either     
	-2.6521	bummer         		2.0723	no prob        
	-2.6246	plz say        		1.9736	not complaining
	-2.6212	not cool       		1.9407	nothing wrong  
	-2.5789	disappointing  		1.9392	congratulations
	-2.5640	gutted         		1.9343	smiling        
	-2.5602	not nice       		1.8951	no need        
	-2.4894	heartbroken    		1.8203	not problem    
	-2.4205	got excited    		1.7726	fuck yeah      
	-2.4099	depressed      		1.7614	thankyou       
	-2.4098	rip            		1.7428	sad sad        
	-2.3734	poor           		1.7254	welcome        
	-2.3571	don love       		1.7252	smile          
	-2.3427	cancelled      		1.6867	no probs       
	-2.3169	unfortunate